In [2]:

# 警告無視
import warnings
warnings.filterwarnings('ignore')

# ライブラリーのインストール
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns # type: ignore
#matplotlibがバックグラウンドで動くの確認
matplotlib.use('Agg')  
plt.switch_backend('Agg')
#スタイルの設定
sns.set(style='whitegrid')

In [3]:
import pandas as pd
# データ読み取り
data_path = 'KO_1919-09-06_2025-04-06.csv'
try:
    # 日付をデータに
    df = pd.read_csv(data_path, parse_dates=['date'])
except Exception as e:
    print(f'Error loading data: {e}')

# サンプルデータ表示
print(f'DataFrame shape: {df.shape}')
df.head()

DataFrame shape: (15922, 7)


,date,open,high,low,close,adj_close,volume
0,1962-01-02 00:00:00-05:00,0.263021,0.270182,0.263021,0.263021,0.046041,806400
1,1962-01-03 00:00:00-05:00,0.259115,0.259115,0.253255,0.257161,0.045016,1574400
2,1962-01-04 00:00:00-05:00,0.257813,0.261068,0.257813,0.259115,0.045358,844800
3,1962-01-05 00:00:00-05:00,0.259115,0.262370,0.252604,0.253255,0.044332,1420800
4,1962-01-08 00:00:00-05:00,0.251302,0.251302,0.245768,0.250651,0.043876,2035200


In [4]:
#データの欠損値の確認
missing_summary = df.isnull().sum
print ('Missing values summary:')
print(missing_summary)
#日付を基本的な日付と時間の型にする
if not pd.api.types.is_datetime64_any_dtype(df['date']):
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
#日付変換に失敗した行を削除
df = df.dropna(subset=['date']) 
#データフレームの確認
print(f'Post-cleaning DataFrame shape: {df.shape}')


Missing values summary:
<bound method DataFrame.sum of         date   open   high    low  close  adj_close  volume
0      False  False  False  False  False      False   False
1      False  False  False  False  False      False   False
2      False  False  False  False  False      False   False
3      False  False  False  False  False      False   False
4      False  False  False  False  False      False   False
...      ...    ...    ...    ...    ...        ...     ...
15917  False  False  False  False  False      False   False
15918  False  False  False  False  False      False   False
15919  False  False  False  False  False      False   False
15920  False  False  False  False  False      False   False
15921  False  False  False  False  False      False   False

[15922 rows x 7 columns]>
Post-cleaning DataFrame shape: (6725, 7)


In [5]:
#数値の特徴を統計的に
print(df.describe())


              open         high          low        close    adj_close  \
count  6725.000000  6725.000000  6725.000000  6725.000000  6725.000000   
mean     16.327344    16.459607    16.190200    16.332802    11.146225   
std      18.730169    18.868148    18.587342    18.736217    15.823639   
min       0.200521     0.203125     0.200521     0.202474     0.036190   
25%       0.837240     0.842448     0.829427     0.835938     0.205131   
50%       5.500000     5.578125     5.437500     5.500000     2.415542   
75%      29.437500    29.750000    29.090000    29.424999    15.236497   
max      72.070000    72.650002    70.730003    72.320000    71.790222   

             volume  
count  6.725000e+03  
mean   8.846602e+06  
std    7.633553e+06  
min    7.680000e+04  
25%    2.611200e+06  
50%    7.439600e+06  
75%    1.290000e+07  
max    1.037760e+08  


In [ ]:
# 数値特徴量の要約統計
print(df.describe())

# 数値特徴のヒストグラムを作成する
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[col], kde=True, bins=30, color='skyblue')
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

#数値列が4つ以上ある場合、相関ヒートマップを作成します。
numeric_df = df.select_dtypes(include=[np.number])
if numeric_df.shape[1] >= 4:
    plt.figure(figsize=(10, 8))
    corr = numeric_df.corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Heatmap')
    plt.tight_layout()
    plt.show()

# 特徴間の関係を視覚化するペアプロット
sns.pairplot(df[numeric_cols])
plt.suptitle('Pair Plot of Numeric Features', y=1.02)
plt.show()

              open         high          low        close    adj_close  \
count  6725.000000  6725.000000  6725.000000  6725.000000  6725.000000   
mean     16.327344    16.459607    16.190200    16.332802    11.146225   
std      18.730169    18.868148    18.587342    18.736217    15.823639   
min       0.200521     0.203125     0.200521     0.202474     0.036190   
25%       0.837240     0.842448     0.829427     0.835938     0.205131   
50%       5.500000     5.578125     5.437500     5.500000     2.415542   
75%      29.437500    29.750000    29.090000    29.424999    15.236497   
max      72.070000    72.650002    70.730003    72.320000    71.790222   

             volume  
count  6.725000e+03  
mean   8.846602e+06  
std    7.633553e+06  
min    7.680000e+04  
25%    2.611200e+06  
50%    7.439600e+06  
75%    1.290000e+07  
max    1.037760e+08  


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression # 今回追加
from sklearn.metrics import r2_score

# # DataFrameが空でなく、十分な行数があるかチェックする
if df.shape[0] == 0:
    print('The DataFrame is empty. Please check the data source and the parsing logic.')
else:
    # 特徴とターゲットを定義する
    features = ['open', 'high', 'low', 'volume']
    target = 'close'

    #  DataFrameからフィーチャーとターゲットを選択する
    X = df[features]
    y = df[target]

    # 訓練とテストの分割を実行するのに十分なサンプルがあることを確認する。   
    if X.shape[0] < 10:
        print('Not enough data to perform training and testing. At least 10 samples are required.')
    else:
        try:
            # 訓練セットが空でないことを保証する訓練サイズでデータを分割する
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
        # トレーニングセットが空かどうかをチェックする
            if X_train.shape[0] == 0:
                raise ValueError('The resulting training set is empty. Adjust the train-test split parameters.')

            # 線形回帰モデルの定義と訓練
            lr_model = LinearRegression()
            lr_model.fit(X_train, y_train)

            # テストセットで予測を行う
            y_pred = lr_model.predict(X_test)
            # R2乗を用いてモデルの性能を評価する。
            r2 = r2_score(y_test, y_pred)
            print(f'R-squared Score for Linear Regression Model: {r2:.4f}')
        except ValueError as ve:
            print(f'ValueError encountered: {ve}')
            print('This error may occur if the dataset does not contain enough samples for the provided test_size. Consider adjusting the parameters or reviewing the data loading steps.')
        except Exception as e:
            print(f'An unexpected error occurred: {e}')
            

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # min/max を使うためにインポート

# (前のセルで y_test, y_pred が計算されていると仮定します)
# y_test: テストデータの実際の値 (pandas Series または NumPy 配列)
# y_pred: モデルが予測した値 (NumPy 配列)

# --- 1. 実績値 vs. 予測値の散布図 ---
print("\n--- 1. 実績値 vs. 予測値の散布図 ---")
plt.figure(figsize=(8, 8)) # グラフのサイズを正方形に近づける
plt.scatter(y_test, y_pred, alpha=0.6, edgecolors='w', linewidth=0.5)

# 完璧な予測を示す対角線 (y=x の線)
# y_test と y_pred の最小値・最大値を取得して線の範囲を決める
all_values = np.concatenate([np.asarray(y_test).flatten(), y_pred.flatten()]) # y_testがSeriesの場合も考慮
min_val = all_values.min()
max_val = all_values.max()
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction Line (y=x)')

plt.xlabel('Actual Values (実績値)')
plt.ylabel('Predicted Values (予測値)')
plt.title('Actual vs. Predicted Values Scatter Plot')
plt.legend()
plt.grid(True)
plt.show()


# --- 2. 時系列での実績値と予測値の比較グラフ ---
print("\n--- 2. 時系列での実績値と予測値の比較グラフ ---")
# このプロットは、y_test のインデックスが元の時系列情報を保持している場合に最も意味があります。
# train_test_split で random_state を指定してシャッフルした場合でも、
# y_test が pandas Series であれば元のインデックスを保持しています。
# そのインデックスでソートすることで時系列順に戻せます。

if isinstance(y_test, pd.Series):
    # y_test が pandas Series の場合、そのインデックスを利用
    results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}, index=y_test.index)
    results_df_sorted = results_df.sort_index() # インデックス（日付など）でソートして時系列順にする
    
    plt.figure(figsize=(14, 7))
    plt.plot(results_df_sorted.index, results_df_sorted['Actual'], label='Actual Values', marker='.', linestyle='-')
    plt.plot(results_df_sorted.index, results_df_sorted['Predicted'], label='Predicted Values', marker='.', linestyle='--')
    plt.xlabel('Time / Index (Sorted)')
    plt.ylabel('Target Value (e.g., Close Price)')
    plt.title('Actual vs. Predicted Values Over Time (Sorted by Index)')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    # y_test が NumPy 配列の場合、時系列インデックスがないため、単純な連番でプロット
    # この場合、元の時系列順序は保証されないため注意が必要です
    print("Warning: y_test is not a pandas Series. Plotting in current order, which might not be chronological if data was shuffled.")
    # (任意) 比較のためにDataFrameを作成
    results_df_numpy = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
    
    plt.figure(figsize=(14, 7))
    plt.plot(results_df_numpy.index, results_df_numpy['Actual'], label='Actual Values (Potentially Shuffled Order)', marker='.', linestyle='-')
    plt.plot(results_df_numpy.index, results_df_numpy['Predicted'], label='Predicted Values (Potentially Shuffled Order)', marker='.', linestyle='--')
    plt.xlabel('Sample Index (Order might be shuffled)')
    plt.ylabel('Target Value (e.g., Close Price)')
    plt.title('Actual vs. Predicted Values (Order as in y_test)')
    plt.legend()
    plt.grid(True)
    plt.show()


--- 1. 実績値 vs. 予測値の散布図 ---

--- 2. 時系列での実績値と予測値の比較グラフ ---
